In [1]:
import numpy as np
import os 

In [2]:
!pwd

/home/shuaman/video_sm/video_summarization/notebooks


In [3]:
os.chdir("/home/shuaman/video_sm/video_summarization")

In [4]:
from src.utils.vsm_dataset import VSMDataset

In [42]:
paths = [
        "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_ovp_processed.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_summe_processed.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_tvsum_processed.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_youtube_processed.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/dataset_cosum_processed.h5",
        ]

In [43]:
features_rn = []
features_iv3 = []
features_3D = []
features= []
features_rgb = []
features_flow= []

for feature in ["features_rn", "features_iv3", "features_3D", "features", "features_rgb", "features_flow"]:
    print(feature)
    for path in paths:
        training_set = VSMDataset(path, googlenet=True, resnext=True, inceptionv3=True,
                                 i3d_rgb=True, i3d_flow=True, resnet3d=True)

        for i in range(len(training_set)):
            feat = training_set.get_feature(i, feature)
            
            for nframe in range(len(feat)):        
                if feature=="features_rn":
                    features_rn.append(feat[nframe])
                elif feature=="features_iv3":
                    features_iv3.append(feat[nframe])
                elif feature=="features_3D":
                    features_3D.append(feat[nframe])
                elif feature=="features":
                    features.append(feat[nframe])
                elif feature=="features_rgb":
                    features_rgb.append(feat[nframe])
                elif feature=="features_flow":
                    features_flow.append(feat[nframe])
                
features_rn = np.array(features_rn)    
features_iv3 = np.array(features_iv3)    
features_3D = np.array(features_3D)    
features = np.array(features)    
features_rgb = np.array(features_rgb)    
features_flow = np.array(features_flow)    

features_rn
features_iv3
features_3D
features
features_rgb
features_flow


In [134]:
feat.shape

(350, 1024)

In [135]:
feat

array([[0.01030507, 0.        , 0.17212544, ..., 0.00933762, 0.14797555,
        0.        ],
       [0.01623172, 0.        , 0.17408632, ..., 0.        , 0.08356722,
        0.01061142],
       [0.00630657, 0.        , 0.16615345, ..., 0.        , 0.03121687,
        0.05343444],
       ...,
       [0.01119477, 0.        , 0.20672494, ..., 0.        , 0.14872962,
        0.00390182],
       [0.01037787, 0.        , 0.15833198, ..., 0.        , 0.16371168,
        0.00138861],
       [0.01265803, 0.        , 0.18552539, ..., 0.        , 0.15395667,
        0.        ]], dtype=float32)

In [139]:
preprocessing.normalize(feat, norm='l2').shape

(350, 1024)

(350, 1024)

In [140]:
features.shape

(69676, 1024)

In [158]:
np.linalg.norm(feat[3])

12.605319

In [160]:
sum((feat[2]/np.linalg.norm(feat[2]))**2)

0.9999999745362396

In [161]:
feat[2]/np.linalg.norm(feat[2])

array([0.00049218, 0.        , 0.01296701, ..., 0.        , 0.00243624,
       0.00417015], dtype=float32)

In [164]:
preprocessing.normalize(feat, norm='l2')[2]

array([0.00049218, 0.        , 0.01296701, ..., 0.        , 0.00243624,
       0.00417015], dtype=float32)

In [166]:
np.linalg.norm(preprocessing.normalize(feat, norm='l2')[2])

1.0

In [171]:
import torch

In [180]:
torch.tensor(feat[0]).shape

torch.Size([1024])

In [176]:
torch.tensor(feat[0]).squeeze().shape

torch.Size([1024])

In [179]:
torch.tensor(feat[0]).unsqueeze(1).shape

torch.Size([1024, 1])

In [165]:
preprocessing.normalize(feat, norm='l2')[2]/np.linalg.norm(preprocessing.normalize(feat, norm='l2')[2])

array([0.00049218, 0.        , 0.01296701, ..., 0.        , 0.00243624,
       0.00417015], dtype=float32)

In [87]:
for i in features:
    if np.sum(i ** 2) <1:
        print("si")

In [94]:
from sklearn import preprocessing

In [88]:
np.sum(features[0] ** 2)

262.1891

In [95]:
features[0]

array([0.22323589, 0.121519  , 0.41137224, ..., 0.29724607, 0.35464266,
       0.66032904], dtype=float32)

In [115]:
features.shape

(69676, 1024)

In [127]:
xd = preprocessing.normalize(features, norm='l2')
xd

array([[0.01378659, 0.00750476, 0.0254055 , ..., 0.0183573 , 0.02190199,
        0.04078055],
       [0.01057878, 0.00453828, 0.02616318, ..., 0.0102624 , 0.03332447,
        0.00807204],
       [0.01823394, 0.0132541 , 0.03702414, ..., 0.00984378, 0.02004703,
        0.03640542],
       ...,
       [0.01170601, 0.01111678, 0.04207801, ..., 0.00758443, 0.01619762,
        0.00732499],
       [0.01538386, 0.00353778, 0.04117977, ..., 0.05245328, 0.04133239,
        0.00459293],
       [0.01529368, 0.00393989, 0.04138784, ..., 0.05307002, 0.04188594,
        0.00431658]], dtype=float32)

In [128]:
preprocessing.normalize(xd, norm='l2')

array([[0.01378659, 0.00750476, 0.0254055 , ..., 0.0183573 , 0.02190199,
        0.04078055],
       [0.01057878, 0.00453828, 0.02616318, ..., 0.0102624 , 0.03332448,
        0.00807204],
       [0.01823394, 0.01325411, 0.03702415, ..., 0.00984378, 0.02004704,
        0.03640543],
       ...,
       [0.01170602, 0.01111678, 0.04207802, ..., 0.00758443, 0.01619763,
        0.00732499],
       [0.01538386, 0.00353778, 0.04117977, ..., 0.05245327, 0.04133239,
        0.00459293],
       [0.01529368, 0.00393989, 0.04138784, ..., 0.05307002, 0.04188594,
        0.00431658]], dtype=float32)

In [156]:
sum(preprocessing.normalize(xd, norm='l2')[0])

22.35702791296535

In [109]:
features[:1].shape

(1, 1024)

In [110]:
preprocessing.normalize(features[:1], norm='l2')[0]

array([0.01378659, 0.00750476, 0.0254055 , ..., 0.0183573 , 0.02190199,
       0.04078055], dtype=float32)

In [111]:
def xdd(xd):
    if xd is not None:
        print("hola")

In [ ]:
a = None

In [45]:
features_rgb.shape

(53872, 1024)

In [89]:
np.sum(features_rgb[0] ** 2)

178.24483

In [46]:
features_flow.shape

(53667, 1024)

In [90]:
np.sum(features_flow[0] ** 2)

127.78471

In [47]:
features_iv3.shape

(69676, 2048)

In [91]:
np.sum(features_iv3[0] ** 2)

527.78986

In [48]:
features_3D.shape

(53872, 2048)

In [92]:
np.sum(features_3D[0] ** 2)

717.1603

In [49]:
features_rn.shape

(69676, 2048)

In [93]:
np.sum(features_rn[0] ** 2)

811.7162

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [14]:
def fit_transform_data(descriptors_orig, n_components):
    """
    descriptors_orig: descriptores directamente obtenidos del modelo
    n_components: numero de componentes de pca
    """
    normalizer = StandardScaler()
    normalizer = normalizer.fit(descriptors_orig)
    descriptors = normalizer.transform(descriptors_orig)
    pca_transformer = PCA(n_components = n_components, random_state=0)
    pca_transformer = pca_transformer.fit(descriptors)
    descriptors = pca_transformer.transform(descriptors)
    return descriptors, normalizer, pca_transformer

In [15]:
def plot_most_imp_feature(features, pca, max_feature_per_comp=1024):
    col_last = []
    try:
        for n_component in range(pca.n_components_):
            col_by_eigenvalues=[]
            val_by_eigen_values=[]
            print("Component # {}".format(n_component+1))
            for pos in np.abs(pca.components_)[n_component].argsort()[::-1]:
                #print(df_empresas_notna[COL_CLUSTER].columns[pos])
                if((np.abs(pca.components_)[n_component][pos]>0.1)):
                    col_by_eigenvalues.append(df_empresas_notna[COL_CLUSTER].columns[pos])
                    val_by_eigen_values.append(pca.components_[n_component][pos])
                    
                    col_last = col_last + [i for i in col_by_eigenvalues[:max_feature_per_comp] if i not in col_last]
                    
                #print(pca.components_[n_component-1][pos])
            print()
            plt.figure(figsize=(15,10))
            plt.title("Feature Coefficient value of component {}".format(n_component+1), fontsize=30)
            plt.xlabel("Coefficient value",fontsize=20)
            plt.ylabel("Feature",fontsize=20)
            sns.barplot(y=col_by_eigenvalues, x=val_by_eigen_values)
            plt.savefig('eigenvalues_pca_n_{}.png'.format(n_component+1))
            plt.show()
    except:
        print("This number component doesnt exist")
    
    return col_last

In [16]:
features_iv3_pca, normalizer_iv3, pca_iv3 = fit_transform_data(descriptors_orig=features_iv3, n_components=1024)
features_3D_pca, normalizer_3D, pca_3D = fit_transform_data(descriptors_orig=features_3D, n_components=1024)
features_rn_pca, normalizer_rn, pca_rn = fit_transform_data(descriptors_orig=features_rn, n_components=1024)

_, normalizer_googlenet, _ = fit_transform_data(descriptors_orig=features, n_components=1024)
_, normalizer_rgb, _ = fit_transform_data(descriptors_orig=features_rgb, n_components=1024)
_, normalizer_flow, _ = fit_transform_data(descriptors_orig=features_flow, n_components=1024)

In [17]:
pca_iv3.explained_variance_ratio_.sum()

0.9658224613377221

In [18]:
pca_3D.explained_variance_ratio_.sum()

0.98189807

In [19]:
pca_rn.explained_variance_ratio_.sum()

0.9747770811544656

In [20]:
features_iv3_pca.shape

(69676, 1024)

In [21]:
features_3D_pca.shape

(53872, 1024)

In [22]:
features_rn_pca.shape

(69676, 1024)

In [23]:
import pickle

In [20]:
transformations = {
    "normalizer_iv3": normalizer_iv3,
    "pca_iv3": pca_iv3,
    "normalizer_3D": normalizer_3D,
    "pca_3D": pca_3D,
    "normalizer_rn": normalizer_rn,
    "pca_rn": pca_rn,
    "normalizer_googlenet": normalizer_googlenet,
    "normalizer_rgb": normalizer_rgb,
    "normalizer_flow": normalizer_flow
}

In [21]:
filename = "transformations.pk"

pickle.dump(transformations, open(filename, 'wb'))

In [23]:
loaded_model = pickle.load(open(filename, 'rb'))

In [24]:
loaded_model

{'normalizer_iv3': StandardScaler(),
 'pca_iv3': PCA(n_components=1024, random_state=0),
 'normalizer_3D': StandardScaler(),
 'pca_3D': PCA(n_components=1024, random_state=0),
 'normalizer_rn': StandardScaler(),
 'pca_rn': PCA(n_components=1024, random_state=0),
 'normalizer_googlenet': StandardScaler(),
 'normalizer_rgb': StandardScaler(),
 'normalizer_flow': StandardScaler()}

In [29]:
features_iv3.shape

(69676, 2048)

In [32]:
features_iv3[-1].reshape(1,-1).shape

(1, 2048)

In [38]:
features_iv3.shape

(69676, 2048)

In [40]:
loaded_model["pca_iv3"].transform(loaded_model["normalizer_iv3"].transform(features_iv3)).shape

(69676, 1024)

# Googlenet,RGB and FLOW original features

In [25]:
paths_original = [
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5",
        ]

paths_i3d_orig = [
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5",
        ]


paths_gn_orig = [
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_ovp_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_summe_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_tvsum_google_pool5.h5",
        "/data/shuaman/video_summarization/datasets/processed_datasets/eccv16_dataset_youtube_google_pool5.h5",
        ]

In [27]:
features_gn_orig = []
features_rgb_orig = []
features_flow_orig = []


training_set = VSMDataset(paths_gn_orig, googlenet=True, resnext=False, inceptionv3=False,
                         i3d_rgb=False, i3d_flow=False, resnet3d=False)

for i in range(len(training_set)):
    feat = training_set.get_feature(i, "features")

    for nframe in range(len(feat)):        
        features_gn_orig.append(feat[nframe])
                
features_gn_orig = np.array(features_gn_orig)



training_set = VSMDataset(paths_i3d_orig, googlenet=False, resnext=False, inceptionv3=False,
                         i3d_rgb=True, i3d_flow=True, resnet3d=False)

for i in range(len(training_set)):
    features_rgb_t = training_set.get_feature(i, "features_rgb")
    features_flow_t = training_set.get_feature(i, "features_flow")
    for nframe in range(len(features_rgb_t)):        
        features_rgb_orig.append(features_rgb_t[nframe])
    for nframe in range(len(features_flow_t)):            
        features_flow_orig.append(features_flow_t[nframe])
                
features_rgb_orig = np.array(features_rgb_orig)
features_flow_orig = np.array(features_flow_orig)





In [182]:
np.sum(features_gn_orig[2] ** 2)

1.000000150419107

In [183]:
features_gn_orig.shape

(55522, 1024)

In [185]:
np.sum(features_rgb_orig[2] ** 2)

2495.5127

In [29]:
features_rgb_orig.shape

(458079, 1024)

In [186]:
np.sum(features_flow_orig[2] ** 2)

128.57715

In [30]:
features_flow_orig.shape

(460902, 1024)

# Comparison GNet with others googlenet

In [31]:
features.shape

(69676, 1024)

In [32]:
features_gn_orig.shape

(55522, 1024)

In [38]:
print(features.std())
print(features.max())
print(features.min())
print(features.mean())

0.35224712
11.704759
0.0
0.35622993


In [39]:
print(features_gn_orig.std())
print(features_gn_orig.max())
print(features_gn_orig.min())
print(features_gn_orig.mean())

0.026327690975322143
0.47543802857398987
0.0
0.016834940797225705


# Comparison IV3 with others IV3

In [51]:
features_rgb.shape

(53872, 1024)

In [50]:
features_rgb_orig.shape

(458079, 1024)

In [52]:
print(features_rgb.std())
print(features_rgb.max())
print(features_rgb.min())
print(features_rgb.mean())

0.2843771
10.579835
0.0
0.23553562


In [53]:
print(features_rgb_orig.std())
print(features_rgb_orig.max())
print(features_rgb_orig.min())
print(features_rgb_orig.mean())

73.3974
1536.6721
-0.26217043
43.720673


In [54]:
features_flow.shape

(53667, 1024)

In [55]:
features_flow_orig.shape

(460902, 1024)

In [56]:
print(features_flow.std())
print(features_flow.max())
print(features_flow.min())
print(features_flow.mean())

0.34659687
4.6505795
0.0
0.14466725


In [57]:
print(features_flow_orig.std())
print(features_flow_orig.max())
print(features_flow_orig.min())
print(features_flow_orig.mean())

8.415225
1272.2615
-0.22121702
1.3784237


In [61]:
!pip3 freeze | wc -l

99
